# Simulation analysis 

In [ ]:
import sys
import os
from os.path import join
from tqdm.notebook import tqdm
from tqdm.notebook import trange
import importlib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import proplot as pplt
from ipywidgets import widgets
from ipywidgets import interactive

from utils import load_bunch
from utils import load_history

sys.path.append('/Users/46h/Research/')
from beamphys import plotting as mplt
from beamphys import utils

pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'
pplt.rc['figure.facecolor'] = 'white'
pplt.rc['savefig.dpi'] = 150
# pplt.rc['pdf.fonttype'] = 42

## Load data

In [ ]:
folder = '/Users/46h/Dropbox (ORNL)/work/btf/simulations/2022-09-20/42mA/'
os.listdir(folder)

In [ ]:
prefix = '220920164350-run_btf-0-HZ04'
node = 'HZ04'

In [ ]:
filenames = {
    'bunch': join(folder, prefix + f'_bunch_{node}.dat'),
    'history': join(folder, prefix + '_history.dat'),
}

In [ ]:
history = load_history(filenames['history'])

In [ ]:
X = load_bunch(filenames['bunch'])
print('X.shape =', X.shape)

## Plot bunch 

In [ ]:
dims = ["x", "x'", "y", "y'", "z", "dE"]
units = ["mm", "mrad", "mm", "mrad", "mm", "keV"]
labels = [f'{d} [{u}]' for d, u in zip(dims, units)]

Corner plot of 2D projections.

In [ ]:
cmap = pplt.Colormap('mono', left=0.065, right=0.9)
cmap

In [ ]:
axes = mplt.corner(
    X, labels=labels, kind='hist',
    mask_zero=True,
    cmap=cmap,
)
axes.format(xlabel_kw=dict(fontsize='large'), ylabel_kw=dict(fontsize='large'))
plt.savefig(f'_output/{prefix}_{node}_corner.png')

Interactive plots.

In [ ]:
prof_kws=dict(lw=0.4, alpha=0.5, color='white', scale=0.09)

mplt.interactive_proj2d_discrete(
    X,
    nbins=33,
    dims=dims,
    units=units,
    prof_kws=prof_kws,
)

5D distribution (beware large grids)

In [ ]:
ind = (0, 1, 2, 3, 4)
_dims = [dims[i] for i in ind]
_units = [units[i] for i in ind]
_labels = [labels[i] for i in ind]

f, edges = np.histogramdd(X[:, ind], bins=34)
f = f / np.max(f)

_coords = [utils.get_bin_centers(e) for e in edges]

In [ ]:
mplt.interactive_proj1d(f, coords=_coords, dims=_dims, units=_units,
                        slice_type='int')

In [ ]:
mplt.interactive_proj2d(f, coords=_coords, dims=_dims, units=_units,
                        slice_type='int')

In [ ]:
axis = (0, 2, 4)
_f = utils.project(f, axis=axis)
_f = _f / np.max(_f)
__coords = [_coords[i] for i in axis]
__dims = [_dims[i] for i in axis]
__dims[-1] = 'w'
__units = [_units[i] for i in axis]
__dims_units = [f'{d} [{u}]' for d, u in zip(__dims, __units)]

ncols = 5
start_stop = [
    (4, len(__coords[0]) - 1 - 4),
    (10, len(__coords[1]) - 1 - 10),
    (25, len(__coords[2]) - 1 - 25),
]

cmap = 'viridis'
for i, ((start, stop), c) in enumerate(zip(start_stop, __coords)):
    fig, axes = pplt.subplots(ncols=ncols, nrows=1, figwidth=6.0)
    jj = np.linspace(start, stop, ncols).astype(int)
    for ax, j in zip(axes, jj):
        idx = utils.make_slice(_f.ndim, i, int(j))
        axis_view = [k for k in range(_f.ndim) if k != i]
        mplt.plot_image(
            _f[idx], 
            x=__coords[axis_view[0]], 
            y=__coords[axis_view[1]], 
            ax=ax,
            vmin=0.0, vmax=1.0,
            linewidth=0, rasterized=True,
            cmap=cmap,
            # colorbar=ax==axes[-1], 
            colorbar_kw=dict(
                width=0.1, 
                # ticks=[0, 1],
            ),
            profx=True, profy=True, prof_kws=dict(lw=0.4, alpha=0.5, color='white', scale=0.09),
        )
        if ax.get_xlim()[1] < ax.get_xlim()[0]:
            ax.format(xlim=sorted(ax.get_xlim()))
        ax.annotate(r"${} \approx {:.0f}$ [{}]".format(__dims[i], __coords[i][j], __units[i]), 
                    color='white', 
                    fontsize='small',
                    xy=(0.02, 0.98), verticalalignment='top', xycoords='axes fraction')
    axes.format(
        xlabel=__dims_units[axis_view[0]], 
        ylabel=__dims_units[axis_view[1]],
    )
    plt.savefig(f'_output/{__dims[axis_view[0]]}{__dims[axis_view[1]]}_{__dims[i]}slice_{cmap}')
    plt.show()

## Plot history

In [ ]:
history.head()

In [ ]:
fig, ax = pplt.subplots(figsize=(3, 2))
ax.plot(history['s'], history['eps_x'] / history.loc[0, 'eps_x'], label=r'$\varepsilon_x$')
ax.plot(history['s'], history['eps_y'] / history.loc[0, 'eps_y'], label=r'$\varepsilon_y$')
ax.plot(history['s'], history['eps_z'] / history.loc[0, 'eps_z'], label=r'$\varepsilon_z$')
ax.legend(ncols=1, handlelength=1.5)
ax.format(xlabel='s [m]', ylabel='Relative emittance growth')
plt.savefig('_output/eps_rms.png')
plt.show()